In [5]:
#Import required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import json
from datetime import datetime
from flask import Flask, render_template, jsonify, request
from werkzeug.utils import secure_filename

app = Flask(__name__, static_folder='static', template_folder='templates')


In [4]:
#Configure variables for adversarial results
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'json', 'csv'}
REPO_PATH = 'adversarial_results' 
GITHUB_URL = 'https://github.com/UniSA-ICT-2025-SP1-P2/capstone_project/tree/master' #replace with correct URL once files created

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(REPO_PATH, exist_ok=True)

NameError: name 'app' is not defined

In [ ]:
#Helper functions
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def load_results(file_path):
    """Load results from JSON or CSV file"""
    if file_path.endswith('.json'):
        with open(file_path, 'r') as f:
            return json.load(f)
    elif file_path.endswith('.csv'):
        import pandas as pd
        return pd.read_csv(file_path).to_dict('records')
    return None

def sync_with_github():
    """Pull latest results from GitHub and push new results"""
    try:
        # Initialize repo if not exists
        if not os.path.exists(os.path.join(REPO_PATH, '.git')):
            repo = git.Repo.init(REPO_PATH)
            origin = repo.create_remote('origin', GITHUB_URL)
        else:
            repo = git.Repo(REPO_PATH)
            origin = repo.remote('origin')
        
        # Pull latest changes
        origin.pull()
        
        # Add all files, commit and push
        repo.git.add(all=True)
        commit_message = f"Update results - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
        repo.git.commit('-m', commit_message)
        origin.push()
        return True
    except Exception as e:
        print(f"GitHub sync error: {str(e)}")
        return False

In [ ]:
#Generate visualisations
def generate_visualisations(results_data):
    """Generate visualisation images from results data"""
    import matplotlib.pyplot as plt
    import numpy as np
    
    output_files = []
    
    # Clean data directory
    viz_dir = os.path.join('static', 'viz')
    os.makedirs(viz_dir, exist_ok=True)
    
    # Accuracy comparison chart
    plt.figure(figsize=(10, 6))
    
    # Extract data for different scenarios
    model_names = []
    standard_acc = []
    pgd_acc = []
    fgsm_acc = []
    
    for result in results_data:
        model_names.append(result.get('model_name', 'Unknown'))
        standard_acc.append(result.get('standard_accuracy', 0))
        pgd_acc.append(result.get('pgd_accuracy', 0))
        fgsm_acc.append(result.get('fgsm_accuracy', 0))
    
    # Plot grouped bar chart
    x = np.arange(len(model_names))
    width = 0.25
    
    plt.bar(x - width, standard_acc, width, label='Standard Accuracy')
    plt.bar(x, pgd_acc, width, label='After PGD Attack')
    plt.bar(x + width, fgsm_acc, width, label='After FGSM Attack')
    
    plt.xlabel('Models')
    plt.ylabel('Accuracy (%)')
    plt.title('Model Performance Comparison')
    plt.xticks(x, model_names, rotation=45)
    plt.legend()
    plt.tight_layout()
    
    # Save the plot
    accuracy_plot = os.path.join(viz_dir, 'accuracy_comparison.png')
    plt.savefig(accuracy_plot)
    output_files.append('viz/accuracy_comparison.png')
    plt.close()
    
    # Attack success rate chart
    plt.figure(figsize=(10, 6))
    
    pgd_success = [100 - acc for acc in pgd_acc]
    fgsm_success = [100 - acc for acc in fgsm_acc]
    
    plt.bar(x - width/2, pgd_success, width, label='PGD Success Rate')
    plt.bar(x + width/2, fgsm_success, width, label='FGSM Success Rate')
    
    plt.xlabel('Models')
    plt.ylabel('Attack Success Rate (%)')
    plt.title('Adversarial Attack Success Rate')
    plt.xticks(x, model_names, rotation=45)
    plt.legend()
    plt.tight_layout()
    
    attack_plot = os.path.join(viz_dir, 'attack_success.png')
    plt.savefig(attack_plot)
    output_files.append('viz/attack_success.png')
    plt.close()
    
    # Robustness improvement after retraining
    if any('original_accuracy' in result for result in results_data):
        plt.figure(figsize=(10, 6))
        
        original_acc = []
        retrained_acc = []
        
        for result in results_data:
            if 'original_accuracy' in result and 'retrained_accuracy' in result:
                original_acc.append(result['original_accuracy'])
                retrained_acc.append(result['retrained_accuracy'])
        
        if original_acc and retrained_acc:
            plt.bar(x - width/2, original_acc, width, label='Before Adversarial Training')
            plt.bar(x + width/2, retrained_acc, width, label='After Adversarial Training')
            
            plt.xlabel('Models')
            plt.ylabel('Accuracy under Attack (%)')
            plt.title('Robustness Improvement After Retraining')
            plt.xticks(x, model_names, rotation=45)
            plt.legend()
            plt.tight_layout()
            
            robustness_plot = os.path.join(viz_dir, 'robustness_improvement.png')
            plt.savefig(robustness_plot)
            output_files.append('viz/robustness_improvement.png')
            plt.close()
    
    return output_files

In [ ]:
#Routes
@app.route('/')
def index():
    """Main page with visualisation dashboard"""
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    """Handle file uploads with results data"""
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400
    
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400
    
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        
        # Also save to Git repo
        repo_file_path = os.path.join(REPO_PATH, 'data', filename)
        os.makedirs(os.path.dirname(repo_file_path), exist_ok=True)
        with open(file_path, 'rb') as src_file:
            with open(repo_file_path, 'wb') as dst_file:
                dst_file.write(src_file.read())
        
        # Load results and generate visualizations
        results_data = load_results(file_path)
        viz_files = generate_visualisations(results_data)
        
        # Sync with GitHub
        sync_success = sync_with_github()
        
        return jsonify({
            'success': True,
            'message': 'File uploaded and processed successfully',
            'github_sync': sync_success,
            'visualisations': viz_files
        })
    
    return jsonify({'error': 'Invalid file type'}), 400

@app.route('/results')
def get_results():
    """Return all results data as JSON"""
    results = []
    
    # Get data from repo
    data_dir = os.path.join(REPO_PATH, 'data')
    if os.path.exists(data_dir):
        for filename in os.listdir(data_dir):
            if allowed_file(filename):
                file_path = os.path.join(data_dir, filename)
                data = load_results(file_path)
                if data:
                    if isinstance(data, list):
                        results.extend(data)
                    else:
                        results.append(data)
    
    return jsonify(results)

@app.route('/sync', methods=['POST'])
def github_sync():
    """Force sync with GitHub repository"""
    success = sync_with_github()
    return jsonify({'success': success})

@app.route('/demo')
def demo():
    """Generate demo data for visualisation"""
    demo_data = [
        {
            'model_name': 'RandonForest',
            'standard_accuracy': 94.5,
            'pgd_accuracy': 45.2,
            'fgsm_accuracy': 60.8,
            'original_accuracy': 45.2,
            'retrained_accuracy': 85.3
        },
        {
            'model_name': 'LogisticRegression',
            'standard_accuracy': 93.7,
            'pgd_accuracy': 42.1,
            'fgsm_accuracy': 58.9,
            'original_accuracy': 42.1,
            'retrained_accuracy': 82.7
        }
    ]
    
    # Save demo data
    demo_file = os.path.join(app.config['UPLOAD_FOLDER'], 'demo_data.json')
    with open(demo_file, 'w') as f:
        json.dump(demo_data, f)
    
    # Generate visualizations
    viz_files = generate_visualisations(demo_data)
    
    return jsonify({
        'success': True,
        'message': 'Demo data generated',
        'visualisations': viz_files
    })
